In [1]:
import datetime
import numpy as np
import collections
import pandas as pd
import matplotlib.pyplot as plt

import postgresemear as sql

In [23]:
CODIGOS_EP = [3064, 3065, 3069, 3070, 
              3071, 3072, 3077, 3078, 
              3081, 3086, 3087, 3088, 
              3092, 3095, 3096, 3103, 3105]

COLUNAS_PDD = ['ID',
            'DataDeReferencia',
            'DataDeOriginiacao',
            'DataDaProximaParcela',
            'DataDeVencimento',
            'DataDeRenegociacao',
            'CPF',
            'CodigoDoProduto',
            'CodigoDoContrato',
            'DiasEmAtraso',
            'QuantidadeDeParcelasAbertas',
            'ValorSaldo',
            'ValorPDD',
            'ValorOperacao',
            'RendaDoCliente',
            'ValorDaProximaParcela',
            'RatingContabilFim']

def query_acionamentos(parametros):
    dados_acionamentos = sql.query_para_pandas('''SELECT 
                              eventos.CPF,
                              eventos.DataDoEvento,
                              acionamentos.DiasEmAtrasoAcionamento
                          FROM acionamentos,eventos
                          WHERE acionamentos.ID = eventos.EventoID
                          AND eventos.EventoTipo = 1
                          AND DataDoEvento BETWEEN %s AND %s''',
                          parametros)
    dados_acionamentos.columns = ['CPF','DataDoEvento','DiasAtraso']
    return dados_acionamentos

def query_pdd(parametros):
    dados_pdd = sql.query_para_pandas('''SELECT 
                          pdd.CPF, pdd.DiasEmAtraso, pdd.CodigoDoProduto
                          FROM pdd
                          WHERE DataDeReferencia = %s
                          ''',
                          parametros)
    dados_pdd.columns = ['CPF','DiasAtraso', 'CodigoDoProduto']   
    #dados_pdd = dados_pdd[dados_pdd.CodigoDoProduto.isin(CODIGOS_EP)] - SO EP
    return dados_pdd


# Atencao - o mesmo CPF pode estar em duas faixas diferentes se ele tiver
# mais de um contrato ativo.
def segmentacao_dias_em_atraso(dados_EP): 
    rol_1 = dados_EP[dados_EP.DiasAtraso <= 5]
    rol_1 = len(set(rol_1.CPF))
    rol_2 = dados_EP[(dados_EP.DiasAtraso >= 6) & (dados_EP.DiasAtraso <= 30)]
    rol_2 = len(set(rol_2.CPF))
    rol_3 = dados_EP[(dados_EP.DiasAtraso >= 31) & (dados_EP.DiasAtraso <= 60)]
    rol_3 = len(set(rol_3.CPF))
    rol_4 = dados_EP[(dados_EP.DiasAtraso >= 61) & (dados_EP.DiasAtraso <= 90)]
    rol_4 = len(set(rol_4.CPF))
    rol_5 = dados_EP[(dados_EP.DiasAtraso >= 91) & (dados_EP.DiasAtraso <= 120)]
    rol_5 = len(set(rol_5.CPF))
    rol_6 = dados_EP[(dados_EP.DiasAtraso >= 121) & (dados_EP.DiasAtraso <= 150)]
    rol_6 = len(set(rol_6.CPF))
    rol_7 = dados_EP[(dados_EP.DiasAtraso >= 151) & (dados_EP.DiasAtraso <= 180)]
    rol_7 = len(set(rol_7.CPF))
    rol_8 = dados_EP[dados_EP.DiasAtraso >= 181]
    rol_8 = len(set(rol_8.CPF))
    return [rol_1, rol_2, rol_3, rol_4, rol_5, rol_6, rol_7, rol_8]
    
def plote_barras_duplas(barra_vermelha, 
                        barra_azul, 
                        nomes_colunas,
                        nome_figura = 'figura_anotada.png'):
    barra_total = [x + y for x,y in zip(barra_vermelha,barra_azul)]
    porcentagens = [str(int(100*y/x)) + '%' for x,y in zip(barra_total,barra_vermelha)]
    fig = plt.figure()
    y_pos = np.arange(len(nomes_colunas))
    plt.bar(y_pos, barra_vermelha, color= 'r')
    plt.bar(y_pos, barra_azul, bottom=barra_vermelha, color='b')
    plt.xticks(y_pos, nomes_colunas)
    anotacao_no_topo(fig, barra_total, nomes_colunas, porcentagens)
    fig.savefig(nome_figura, dpi=150)
    plt.close()

def anotacao_no_topo(fig, totalHeight, labels, allPercentages):
    ax = fig.add_subplot(111)
    up = max(totalHeight) * .05
    ax.set_ylim(0, max(totalHeight) + 3 * up)
    y_pos = np.arange(len(labels))
    for xi, yi, l in zip(*[y_pos, totalHeight, list(map(str, allPercentages))]):
        ax.text(xi - len(l) * .05, yi + up, l, bbox=dict(facecolor='w', edgecolor='w', alpha=.5))

def limita_acionamentos(cpfs_referencia, acionamentos):
    nao_encontrados = []
    for i in acionamentos.index:
        if acionamentos.loc[i,'CPF'] not in cpfs_referencia:
            nao_encontrados.append(i)
    return acionamentos.drop(nao_encontrados)        
        

In [8]:
parametros = (datetime.datetime(2019,3,31),)
pdd_marco = query_pdd(parametros)
pdd_marco = pdd_marco[pdd_marco.CodigoDoProduto.isin(CODIGOS_EP)].copy()
parametros = (datetime.datetime(2019,4,30),)
pdd_abril = query_pdd(parametros)
pdd_abril = pdd_abril[pdd_abril.CodigoDoProduto.isin(CODIGOS_EP)].copy()
parametros = (datetime.datetime(2019,5,31),)
pdd_maio = query_pdd(parametros)
pdd_maio = pdd_maio[pdd_maio.CodigoDoProduto.isin(CODIGOS_EP)].copy()
parametros = (datetime.datetime(2019,6,30),)
pdd_junho = query_pdd(parametros)
pdd_junho = pdd_junho[pdd_junho.CodigoDoProduto.isin(CODIGOS_EP)].copy()

In [24]:
parametros = (datetime.datetime(2019,3,1), datetime.datetime(2019,3,31,23,59,59))
acionamentos_marco = query_acionamentos(parametros)
acionamentos_marco = limita_acionamentos(set(pdd_marco.CPF), acionamentos_marco)
parametros = (datetime.datetime(2019,4,1), datetime.datetime(2019,4,30,23,59,59))
acionamentos_abril = query_acionamentos(parametros)
acionamentos_abril = limita_acionamentos(set(pdd_abril.CPF), acionamentos_abril)
parametros = (datetime.datetime(2019,5,1), datetime.datetime(2019,5,31,23,59,59))
acionamentos_maio = query_acionamentos(parametros)
acionamentos_maio = limita_acionamentos(set(pdd_maio.CPF), acionamentos_maio)
parametros = (datetime.datetime(2019,6,1), datetime.datetime(2019,6,30,23,59,59))
acionamentos_junho = query_acionamentos(parametros)
acionamentos_junho = limita_acionamentos(set(pdd_junho.CPF), acionamentos_junho)

In [25]:
dados_acion = []
dados_acion.append(segmentacao_dias_em_atraso(acionamentos_marco))
dados_acion.append(segmentacao_dias_em_atraso(pdd_marco))
dados_acion.append(segmentacao_dias_em_atraso(acionamentos_abril))
dados_acion.append(segmentacao_dias_em_atraso(pdd_abril))
dados_acion.append(segmentacao_dias_em_atraso(acionamentos_maio))
dados_acion.append(segmentacao_dias_em_atraso(pdd_maio))
dados_acion.append(segmentacao_dias_em_atraso(acionamentos_junho))
dados_acion.append(segmentacao_dias_em_atraso(pdd_junho))
pd.DataFrame(data=dados_acion).to_csv('dados_de_acionamentos.csv',sep=';')

In [31]:
barra_vermelha = segmentacao_dias_em_atraso(acionamentos_marco)
total = segmentacao_dias_em_atraso(pdd_marco)
barra_azul = [x - y for x, y in zip(total, barra_vermelha)]
NOMES_COLUNAS = ['-5','6-30','31-60','61-90','\n91-120','121-150','\n151-180','181+']
plote_barras_duplas(barra_vermelha[1:], barra_azul[1:], NOMES_COLUNAS[1:])

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\ipykernel_launcher.py:86: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [30]:
barra_vermelha = segmentacao_dias_em_atraso(acionamentos_abril)
total = segmentacao_dias_em_atraso(pdd_abril)
barra_azul = [x - y for x, y in zip(total, barra_vermelha)]
NOMES_COLUNAS = ['-5','6-30','31-60','61-90','\n91-120','121-150','\n151-180','181+']
plote_barras_duplas(barra_vermelha[1:], barra_azul[1:], NOMES_COLUNAS[1:])

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\ipykernel_launcher.py:86: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [28]:
barra_vermelha = segmentacao_dias_em_atraso(acionamentos_maio)
total = segmentacao_dias_em_atraso(pdd_maio)
barra_azul = [x - y for x, y in zip(total, barra_vermelha)]
NOMES_COLUNAS = ['-5','6-30','31-60','61-90','\n91-120','121-150','\n151-180','181+']
plote_barras_duplas(barra_vermelha[1:], barra_azul[1:], NOMES_COLUNAS[1:])

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\ipykernel_launcher.py:86: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [32]:
barra_vermelha = segmentacao_dias_em_atraso(acionamentos_junho)
total = segmentacao_dias_em_atraso(pdd_junho)
barra_azul = [x - y for x, y in zip(total, barra_vermelha)]
NOMES_COLUNAS = ['-5','6-30','31-60','61-90','\n91-120','121-150','\n151-180','181+']
plote_barras_duplas(barra_vermelha[1:], barra_azul[1:], NOMES_COLUNAS[1:])

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\ipykernel_launcher.py:86: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
